In [8]:
import torch
import torch.nn as nn

# 定义一个通用的、可定制的全连接神经网络（MLP）类
class MLP(nn.Module):
    def __init__(self, layer_dims):
        # 调用父类 nn.Module 的构造函数
        super(MLP, self).__init__()

        # 定义一个空的 nn.ModuleList 来存放所有的线性层
        self.layers = nn.ModuleList()
        # 定义激活函数
        self.activation = nn.Tanh() # 也可以使用 nn.ReLU(), nn.Sigmoid() 等
        # 根据 layer_dims 列表自动创建所有的线性层
        # layer_dims 示例: [2, 32, 32, 1]
        # 循环从第一个维度到倒数第二个维度
        for i in range(len(layer_dims) - 1):
            # 创建一个线性层，输入维度为 layer_dims[i]，输出维度为 layer_dims[i+1]
            layer = nn.Linear(layer_dims[i], layer_dims[i + 1])
            # 将创建的线性层添加到 layers 列表中
            self.layers.append(layer)
    
    def forward(self, x):
        # 这个方法接收输入张量 x。
        # 使用一个循环，让 x 依次通过 nn.ModuleList 中的线性层和激活函数。
        # 注意：最后一层后面通常没有激活函数，因为我们希望输出是原始的数值。
        # 你需要确保你的循环逻辑能正确处理这一点。
        
        # 遍历除了最后一层之外的所有层
        for i, layer in enumerate(self.layers[:-1]): # self.layers[:-1] 表示除了最后一个元素之外的所有元素
            x = layer(x) # 数据通过线性层
            x = self.activation(x) # 线性层后接激活函数
            
        # 处理最后一层（通常没有激活函数）
        x = self.layers[-1](x) # 数据通过最后一个线性层
        
        return x # 返回最终输出


In [17]:
if __name__ == "__main__":
    print("--- MLP 网络构建与测试 ---")
    # 定义一个层级结构示例：输入2个神经元，两个隐藏层各有32个神经元，输出1个神经元
    if torch.cuda.is_available():
        print("CUDA 可用，使用 GPU 进行计算")
        device = torch.device("cuda")
    else:
        device = torch.device("cpu")
        print("CUDA 不可用，使用 CPU 进行计算")
    # 定义网络层级结构
    layers = [2, 32, 32, 1]
    # 实例化你的 MLP 网络
    my_network = MLP(layers)
    # 将网络移动到指定的设备（GPU 或 CPU）
    my_network.to(device)
    # 打印网络结构
    print(f"网络结构: {my_network}")    
    print(f"网络所在的设备: {next(my_network.parameters()).device}") # 打印网络参数所在的设备

    # 创建一个假的输入张量，形状要匹配网络的输入维度
    # 例如，torch.randn(10, 2) 代表10个2维的输入点
    dummy_input = torch.randn(10, layers[0]) # layers[0] 是输入维度
    dummy_input = dummy_input.to(device) # 将假输入移动到同一设备
    print(f"\n输入数据所在的设备: {dummy_input.device}")
    # 将假输入送入网络，得到输出
    output = my_network(dummy_input) # 取消注释，现在 forward 方法已实现
    print(f"输出形状: {output.shape}") 

--- MLP 网络构建与测试 ---
CUDA 可用，使用 GPU 进行计算
网络结构: MLP(
  (layers): ModuleList(
    (0): Linear(in_features=2, out_features=32, bias=True)
    (1): Linear(in_features=32, out_features=32, bias=True)
    (2): Linear(in_features=32, out_features=1, bias=True)
  )
  (activation): Tanh()
)
网络所在的设备: cuda:0

输入数据所在的设备: cuda:0
输出形状: torch.Size([10, 1])


| .to(device)理由           | 一句话                                     |
| ------------ | --------------------------------------- |
| **显式控制**     | 动态图逐行执行，开发者必须明确张量/参数在哪。                 |
| **调试透明**     | `device` 不匹配立刻报错，定位简单。                  |
| **多 GPU 并行** | 手动放置才能做模型并行、数据并行等高级策略。                  |
| **性能优化**     | `.to(device)` 提醒一次性搬数据，避免频繁 CPU↔GPU 拷贝。 |


| 构件 / 概念                              | 一句话重点                                              |
| ------------------------------------ | -------------------------------------------------- |
| `class MLP(nn.Module)`               | 继承 `nn.Module` 获得 PyTorch 模块所有核心能力。                |
| `__init__`                           | 构造函数，**自动**初始化网络层、激活函数等结构。                         |
| `super().__init__()`                 | **必须**先调用，确保父类完成内部初始化。                             |
| `nn.ModuleList()`                    | 用于保存子层，**让 PyTorch 正确追踪参数**。                       |
| `nn.Tanh()`                          | **类**，实例化后作为激活函数。                                  |
| `for` 循环在 `__init__`                 | 根据 `layer_dims` **动态**堆叠线性层与激活。                    |
| `forward(self, x)`                   | 定义前向传播逻辑，**必须实现**。                                 |
| `my_network(x)`                      | **无需**显式调用 `forward()`，PyTorch 通过 `__call__` 自动执行。 |
| `if __name__ == "__main__":`         | 保证测试代码仅在**直接运行**文件时执行。                             |
| `torch.randn(batch_size, input_dim)` | 生成**假数据**，快速验证模型初始化与前向传播。                          |
